# Vežbe 3: Lekser

[Lekser](https://en.wikipedia.org/wiki/Lexical_analysis) jer deo kompajlera koji na osnovu izvornog koda formira niz tokena. Token je uređeni par (klasa, leksema). Leksema je karakter ili ključna reč koja ima funkciju u sintaksi programskog jezika. Upravo ta funkcija određuje klasu lekseme. Lekser će pročitati izvorni kod i to je jedini put kada će se to uraditi u čitavom procesu kompajliranja. Naredne faze kompajliranja zahtevaju samo formirani niz tokena.

![pp-01](https://i.postimg.cc/SNmFQ6X0/pp-01.png)

In [ ]:
from enum import Enum, auto

Klasa **Class** definiše sve moguće klase leksema koje se mogu naći u izvornom kodu.

In [ ]:
class Class(Enum):
    PLUS = auto()
    MINUS = auto()
    STAR = auto()
    FWDSLASH = auto()
    PERCENT = auto()

    OR = auto()
    AND = auto()
    NOT = auto()

    EQ = auto()
    NEQ = auto()
    LT = auto()
    GT = auto()
    LTE = auto()
    GTE = auto()

    LPAREN = auto()
    RPAREN = auto()
    LBRACKET = auto()
    RBRACKET = auto()
    LBRACE = auto()
    RBRACE = auto()

    ASSIGN = auto()
    SEMICOLON = auto()
    COMMA = auto()

    TYPE = auto()
    INT = auto()
    CHAR = auto()
    STRING = auto()

    IF = auto()
    ELSE = auto()
    WHILE = auto()
    FOR = auto()

    BREAK = auto()
    CONTINUE = auto()
    RETURN = auto()
    
    ADDRESS = auto()

    ID = auto()
    EOF = auto()

Klasa **Token** predstavlja uređeni par (klasa, leksema).

Medota **str** vraća string reprezentaciju tokena koja se koristi u procesu pronalaženja grešaka.

In [ ]:
class Token:
    def __init__(self, class_, lexeme):
        self.class_ = class_
        self.lexeme = lexeme

    def __str__(self):
        return "<{} {}>".format(self.class_, self.lexeme)

Klasa **Lekser** sadrži metode za leksičku analizu izvornog koda.

Metoda **lex** formira niz tokena pozivajući metodu **next_token**.

Metoda **next_token** konstruiše token odgovarajuće klase pozivajući metodu **next_char**.

Metoda **next_char** pomera pokazivač na sledeći karakter.

Metoda **read_keyword** konstruiše token ključne reči pod uslovom da je trenutni karakter slovo.

Metoda **read_string** konstruiše token string literala pod uslovom da je trenutni karakter znak navodnika.

Metoda **read_char** konstruiše token literala karaktera pod uslovom da je trenutni karakter apostrof.

Metoda **read_int** konstruiše token literala celog broja pod uslovom da je trenutni karakter cifra.

Metoda **read_space** ne konstruiše token, ali pomera pokazivač na prvi sledeći karakter koji nije razmak.

Metoda **die** se koristi u slučaju da je lekser pročitao neočekivani karakter.

In [ ]:
class Lexer:
    def __init__(self, text):
        self.text = text
        self.len = len(text)
        self.pos = -1

    def read_space(self):
        while self.pos + 1 < self.len and self.text[self.pos + 1].isspace():
            self.next_char()

    def read_int(self):
        lexeme = self.text[self.pos]
        while self.pos + 1 < self.len and self.text[self.pos + 1].isdigit():
            lexeme += self.next_char()
        return int(lexeme)

    def read_char(self):
        self.pos += 1
        lexeme = self.text[self.pos]
        self.pos += 1
        return lexeme

    def read_string(self):
        lexeme = ''
        while self.pos + 1 < self.len and self.text[self.pos + 1] != '"':
            lexeme += self.next_char()
        self.pos += 1
        return lexeme

    def read_keyword(self):
        lexeme = self.text[self.pos]
        while self.pos + 1 < self.len and self.text[self.pos + 1].isalnum():
            lexeme += self.next_char()
        if lexeme == 'if':
            return Token(Class.IF, lexeme)
        elif lexeme == 'else':
            return Token(Class.ELSE, lexeme)
        elif lexeme == 'while':
            return Token(Class.WHILE, lexeme)
        elif lexeme == 'for':
            return Token(Class.FOR, lexeme)
        elif lexeme == 'break':
            return Token(Class.BREAK, lexeme)
        elif lexeme == 'continue':
            return Token(Class.CONTINUE, lexeme)
        elif lexeme == 'return':
            return Token(Class.RETURN, lexeme)
        elif lexeme == 'int' or lexeme == 'char' or lexeme == 'void':
            return Token(Class.TYPE, lexeme)
        return Token(Class.ID, lexeme)

    def next_char(self):
        self.pos += 1
        if self.pos >= self.len:
            return None
        return self.text[self.pos]

    def next_token(self):
        self.read_space()
        curr = self.next_char()
        if curr is None:
            return Token(Class.EOF, curr)
        token = None
        if curr.isalpha():
            token = self.read_keyword()
        elif curr.isdigit():
            token = Token(Class.INT, self.read_int())
        elif curr == '\'':
            token = Token(Class.CHAR, self.read_char())
        elif curr == '"':
            token = Token(Class.STRING, self.read_string())
        elif curr == '+':
            token = Token(Class.PLUS, curr)
        elif curr == '-':
            token = Token(Class.MINUS, curr)
        elif curr == '*':
            token = Token(Class.STAR, curr)
        elif curr == '/':
            token = Token(Class.FWDSLASH, curr)
        elif curr == '%':
            token = Token(Class.PERCENT, curr)
        elif curr == '&':
            curr = self.next_char()
            if curr == '&':
                token = Token(Class.AND, '&&')
            else:
                token = Token(Class.ADDRESS, '&')
                self.pos -= 1
        elif curr == '|':
            curr = self.next_char()
            if curr == '|':
                token = Token(Class.OR, '||')
            else:
                self.die(curr)
        elif curr == '!':
            curr = self.next_char()
            if curr == '=':
                token = Token(Class.NEQ, '!=')
            else:
                token = Token(Class.NOT, '!')
                self.pos -= 1
        elif curr == '=':
            curr = self.next_char()
            if curr == '=':
                token = Token(Class.EQ, '==')
            else:
                token = Token(Class.ASSIGN, '=')
                self.pos -= 1
        elif curr == '<':
            curr = self.next_char()
            if curr == '=':
                token = Token(Class.LTE, '<=')
            else:
                token = Token(Class.LT, '<')
                self.pos -= 1
        elif curr == '>':
            curr = self.next_char()
            if curr == '=':
                token = Token(Class.GTE, '>=')
            else:
                token = Token(Class.GT, '>')
                self.pos -= 1
        elif curr == '(':
            token = Token(Class.LPAREN, curr)
        elif curr == ')':
            token = Token(Class.RPAREN, curr)
        elif curr == '[':
            token = Token(Class.LBRACKET, curr)
        elif curr == ']':
            token = Token(Class.RBRACKET, curr)
        elif curr == '{':
            token = Token(Class.LBRACE, curr)
        elif curr == '}':
            token = Token(Class.RBRACE, curr)
        elif curr == ';':
            token = Token(Class.SEMICOLON, curr)
        elif curr == ',':
            token = Token(Class.COMMA, curr)
        else:
            self.die(curr)
        return token

    def lex(self):
        tokens = []
        while True:
            curr = self.next_token()
            tokens.append(curr)
            if curr.class_ == Class.EOF:
                break
        return tokens

    def die(self, char):
        raise SystemExit("Unexpected character: {}".format(char))

Testiranje implementacije

In [ ]:
test_id = 1
path = f'/content/drive/My Drive/PP/data/test/test{test_id}.c'

with open(path, 'r') as source:
    text = source.read()

    lexer = Lexer(text)
    tokens = lexer.lex()

    for t in tokens:
        print(t)